In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

#Modeling Tools
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
from statsmodels.formula.api import ols

from datetime import date
from scipy import stats


#Custom functions
from env import host, user, password #Database credentials

import wrangle3
import env
import eval_model
## Evaluation tools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

In [2]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test=wrangle3.wrangle()



In [3]:
X_train.head()

,Unnamed: 0,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,county,age,age_bin,taxrate,acres,acres_bin,sqft_bin,structure_dollar_per_sqft,structure_dollar_sqft_bin,land_dollar_per_sqft,lot_dollar_sqft_bin,bath_bed_ratio,cola,logerror_bins,baseline,scaled_latitude,scaled_longitude,scaled_bathroomcnt,scaled_taxrate,scaled_bedroomcnt,scaled_lotsizesquarefeet,scaled_age,scaled_acres,scaled_bath_bed_ratio,scaled_calculatedfinishedsquarefeet,area_cluster,size_cluster,price_cluster,area_cluster_la_newer,area_cluster_la_older,area_cluster_northwest_costal,area_cluster_palmdale_landcaster,area_cluster_santa_clarita,area_cluster_se_coast,size_cluster_a,size_cluster_b,size_cluster_c,size_cluster_d,size_cluster_e,size_cluster_f,size_cluster_g,price_cluster_a,price_cluster_b,price_cluster_c,price_cluster_d,price_cluster_e
21580,21580,11204960,3.0,4.0,2000.0,6037.0,34.561695,-118.082751,6999.0,40227.0,3101.0,97328.0,1987.0,104877.0,131090.0,26213.0,2623.93,Los_Angeles,30.0,0.200,2.001625,0.160675,0.2,0.4,52.438500,0.2,3.745249,0.1,0.750000,0,"(-0.05, 0.05]",0.017913,0.829634,0.731775,0.333333,0.203380,0.500000,0.008825,0.213235,0.008825,0.196429,0.172850,palmdale_landcaster,f,c,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
32781,32781,14323862,6.0,6.0,7393.0,6059.0,33.601524,-117.845231,24000.0,40009.0,1286.0,96975.0,1995.0,1556016.0,2449943.0,893927.0,29921.84,Orange,22.0,0.200,1.221328,0.550964,0.4,0.9,210.471527,0.6,37.246958,0.3,1.000000,0,"(0.05, 0.2]",0.017913,0.177142,0.856593,0.833333,0.123723,0.833333,0.031010,0.154412,0.031010,0.285714,0.741252,se_coast,d,a,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
33161,33161,12205584,2.0,3.0,1778.0,6037.0,33.966002,-118.298627,4948.0,12447.0,3101.0,96025.0,1936.0,87000.0,347000.0,260000.0,4248.21,Los_Angeles,81.0,0.600,1.224268,0.113590,0.1,0.4,48.931384,0.1,52.546483,0.4,0.666667,1,"(-0.05, 0.05]",0.017913,0.424826,0.618331,0.166667,0.124023,0.333333,0.006149,0.588235,0.006149,0.166667,0.149452,la_older,b,d,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
45785,45785,12078527,3.0,4.0,2777.0,6037.0,34.166013,-118.218530,9989.0,45457.0,3101.0,96327.0,1990.0,322342.0,875355.0,553013.0,9603.39,Los_Angeles,27.0,0.200,1.097085,0.229316,0.2,0.6,116.075621,0.4,55.362198,0.4,0.750000,0,"(0.05, 0.2]",0.017913,0.560745,0.660422,0.333333,0.111039,0.500000,0.012727,0.191176,0.012727,0.196429,0.254743,santa_clarita,f,a,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0
45820,45820,14266520,3.0,4.0,1939.0,6059.0,33.706702,-117.898677,6028.0,47568.0,1286.0,97004.0,1971.0,139321.0,327623.0,188302.0,3923.98,Orange,46.0,0.333,1.197712,0.138384,0.1,0.4,71.851986,0.2,31.237890,0.3,0.750000,0,"(-0.05, 0.05]",0.017913,0.248617,0.828507,0.333333,0.121312,0.500000,0.007558,0.330882,0.007558,0.196429,0.166421,se_coast,f,d,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [4]:
X_validate.head()

,Unnamed: 0,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,county,age,age_bin,taxrate,acres,acres_bin,sqft_bin,structure_dollar_per_sqft,structure_dollar_sqft_bin,land_dollar_per_sqft,lot_dollar_sqft_bin,bath_bed_ratio,cola,baseline,scaled_latitude,scaled_longitude,scaled_bathroomcnt,scaled_taxrate,scaled_bedroomcnt,scaled_lotsizesquarefeet,scaled_age,scaled_acres,scaled_bath_bed_ratio,scaled_calculatedfinishedsquarefeet,area_cluster,size_cluster,price_cluster,area_cluster_la_newer,area_cluster_la_older,area_cluster_northwest_costal,area_cluster_palmdale_landcaster,area_cluster_santa_clarita,area_cluster_se_coast,size_cluster_a,size_cluster_b,size_cluster_c,size_cluster_d,size_cluster_e,size_cluster_f,size_cluster_g,price_cluster_a,price_cluster_b,price_cluster_c,price_cluster_d,price_cluster_e
20430,20430,11304460,3.0,4.0,2334.0,6037.0,34.687739,-118.184182,6631.0,5534.0,3101.0,97319.0,1990.0,173161.0,216450.0,43289.0,3463.67,Los_Angeles,27.0,0.200,1.600217,0.152227,0.2,0.5,74.190660,0.2,6.528276,0.2,0.750000,0,0.017913,0.915288,0.678472,0.333333,0.162402,0.500000,0.008345,0.191176,0.008345,0.196429,0.208052,palmdale_landcaster,f,c,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
36350,36350,14260987,2.0,4.0,1361.0,6059.0,33.729501,-117.905911,7800.0,47568.0,1286.0,97004.0,1960.0,62078.0,382037.0,319959.0,4531.62,Orange,57.0,0.400,1.186173,0.179063,0.2,0.3,45.612050,0.1,41.020385,0.3,0.500000,0,0.017913,0.264110,0.824705,0.166667,0.120134,0.500000,0.009870,0.411765,0.009870,0.107143,0.105502,la_newer,c,d,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
24457,24457,14066920,2.5,4.0,1840.0,6059.0,33.710168,-118.023301,6305.0,25218.0,1286.0,96967.0,1965.0,126210.0,711616.0,585406.0,7920.96,Orange,52.0,0.400,1.113095,0.144743,0.1,0.4,68.592391,0.2,92.847898,0.4,0.625000,0,0.017913,0.250972,0.763016,0.250000,0.112674,0.500000,0.007920,0.375000,0.007920,0.151786,0.155987,la_newer,b,a,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
47718,47718,11236044,2.0,3.0,1244.0,6037.0,34.544534,-118.070681,7040.0,40227.0,3101.0,97328.0,1993.0,114200.0,199000.0,84800.0,3453.14,Los_Angeles,24.0,0.200,1.735246,0.161616,0.2,0.2,91.800643,0.3,12.045455,0.2,0.666667,0,0.017913,0.817972,0.738118,0.166667,0.176186,0.333333,0.008879,0.169118,0.008879,0.166667,0.093170,palmdale_landcaster,b,c,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
46236,46236,11317435,3.0,4.0,2200.0,6037.0,34.680030,-118.243096,7759.0,5534.0,3101.0,97319.0,2005.0,204300.0,273000.0,68700.0,4280.00,Los_Angeles,12.0,0.133,1.567766,0.178122,0.2,0.5,92.863636,0.3,8.854234,0.2,0.750000,0,0.017913,0.910049,0.647512,0.333333,0.159089,0.500000,0.009817,0.080882,0.009817,0.196429,0.193929,palmdale_landcaster,f,c,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [5]:
#understanding our cluster features and what the baseline would be
cluster1 = train.groupby(by=['latitude', 'longitude', 'age']).logerror.mean()
cluster1.mean()

0.017966034256587684

In [6]:
cluster2 = train.groupby(by= ['bathroomcnt', 'sqft_bin', 'acres_bin', 'bath_bed_ratio']).logerror.mean()
cluster2.mean()

0.027967956481878043

In [7]:
cluster3 = train.groupby(by=['taxrate', 'structure_dollar_sqft_bin', 'lot_dollar_sqft_bin']).logerror.mean()
cluster3.mean()

0.01793960663089426

In [8]:
#adding in log_error
train2 = X_train
train2['log_error']=y_train
train=train2

In [9]:
# Create Model
ols_model4 = ols(formula='logerror ~ taxvaluedollarcnt + structuretaxvaluedollarcnt + taxvaluedollarcnt + landtaxvaluedollarcnt + acres + age', data=train).fit()

# Make predictions
ols4_yhat = ols_model4.predict(X_train)
ols_eval['ols4_yhat']=ols4_yhat
X_train['ols4_prdictions']=ols4_yhat

NameError: name 'ols_eval' is not defined

In [ ]:
# Create Model
ols_model1 = ols(formula='log_error ~ area_cluster_la_newer + area_cluster_la_older + area_cluster_northwest_costal + area_cluster_palmdale_landcaster + area_cluster_santa_clarita + area_cluster_se_coast + size_cluster_a + size_cluster_b + size_cluster_c + size_cluster_d + size_cluster_e + size_cluster_f + size_cluster_g + price_cluster_a + price_cluster_b + price_cluster_c + price_cluster_d + price_cluster_e', data=train).fit()

# Make predictions
ols1_yhat = ols_model1.predict(X_train)
ols_eval['ols1_yhat']=ols1_yhat
X_train['mvp_prdictions']=ols_yhat


In [12]:
# Create Model
ols_model1 = ols(formula='logerror ~ area_cluster_la_newer + area_cluster_la_older + area_cluster_northwest_costal + area_cluster_palmdale_landcaster + area_cluster_santa_clarita + area_cluster_se_coast + size_cluster_a + size_cluster_b + size_cluster_c + size_cluster_d + size_cluster_e + size_cluster_f + size_cluster_g + price_cluster_a + price_cluster_b + price_cluster_c + price_cluster_d + price_cluster_e', data=train).fit()

# Make predictions
ols_yhat = ols_model1.predict(X_train)
ols_eval['ols_yhat'] = ols_yhat
X_train['mvp_prdictions']=ols_yhat

#  DataFrame for evaluating model 
ols_eval = y_train.copy()
validate_eval = y_validate.copy()
ols_eval = pd.DataFrame(ols_eval)
validate_eval = pd.DataFrame(validate_eval)
ols_eval.rename(columns={'logerror': 'actual'}, inplace=True)
validate_eval.rename(columns={'logerror': 'actual'}, inplace=True)
validate_eval['ols_yhat'] = ols_model1.predict(X_validate)


# Add baseline - median home value
ols_eval['baseline_yhat'] = ols_eval['actual'].mean()
validate_eval['baseline_yhat'] = ols_eval['actual'].mean()
validate_eval['ols_yhat'] = ols_model1.predict(X_validate)

#  Compute the RMSE  and R2 for  ols model and baseline 
baseline_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))

pct_change=round(((ols_RMSE-baseline_RMSE)/baseline_RMSE)*100, 2)
rmse_validate = (sqrt(mean_squared_error(validate_eval.actual, validate_eval.ols_yhat)))
baseline_r2 = (r2_score(ols_eval.actual, ols_eval.baseline_yhat))
ols_train_r2 = (r2_score(ols_eval.actual, ols_eval.ols_yhat))
ols_validate_r2 = (r2_score(validate_eval.actual, validate_eval.ols_yhat))

#Output Findings

print(f'My model has value: {ols_RMSE < baseline_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'My model train RMSE: {ols_RMSE}')
print(f'My model validate RMSE: {rmse_validate}')
print(f'RMSE difference baseline to model train: {baseline_RMSE- ols_RMSE}')
print(f'RMSE difference baseline to model validate: {baseline_RMSE- rmse_validate}')
print(f'RMSE difference train to validate: {ols_RMSE- rmse_validate}')
print(f'RMSE improvement: {pct_change}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Model train  R2: {ols_train_r2}')
print(f'Model Validate R2: {ols_validate_r2}')

NameError: name 'ols_eval' is not defined

In [ ]:
#Calculate Evaluation metrics
baseline_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))
ols1_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols1_yhat)))
pct_change_baseline=(((ols_RMSE-baseline_RMSE)/baseline_RMSE)*100)
pct_change_last_model=(((ols1_RMSE-ols_RMSE)/ols_RMSE)*100)
#rmse_validate = (sqrt(mean_squared_error(validate_eval.actual, validate_eval.ols2_yhat)))
baseline_r2 = (r2_score(ols_eval.actual, ols_eval.baseline_yhat))
ols3_train_r2 = (r2_score(ols_eval.actual, ols_eval.ols1_yhat))
#ols2_validate_r2 = (r2_score(validate_eval.actual, validate_eval.ols2_yhat))
# Display findings
print(f'My model has value: {ols_RMSE < baseline_RMSE}')
print(f'My model beats previous model: {ols1_RMSE < ols_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'Model 1 RMSE: {ols_RMSE}')
print(f'Current model train RMSE: {ols1_RMSE}')
#print(f'Currennt model validate RMSE: {rmse_validate}')
print()
print(f'Current model RMSE difference from baseline: {baseline_RMSE- ols_RMSE}')
#print(f'RMSE difference train to validate: {ols2_RMSE- rmse_validate}')
print(f'Current model baseline RMSE improvement: {pct_change_baseline}%')
#print(f'Current model RMSE improvement from last model: {pct_change_last_model}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Model train  R2: {ols1_train_r2}')
#print(f'Model Validate R2: {ols2_validate_r2}')

In [13]:
ols_eval

NameError: name 'ols_eval' is not defined

In [14]:

# Add baseline - median home value
ols_eval['baseline_yhat'] = ols_eval['actual'].mean()
validate_eval['baseline_yhat'] = ols_eval['actual'].mean()

NameError: name 'ols_eval' is not defined

In [15]:
# Y-Y-y-y
ols_eval

NameError: name 'ols_eval' is not defined

In [16]:
ols_eval['ols_yhat'] = ols_model1.predict(X_train)


NameError: name 'ols_eval' is not defined

In [17]:
validate_eval['ols_yhat'] = ols_model1.predict(X_validate)

NameError: name 'validate_eval' is not defined

In [18]:
# Calculate and Add Residuals Column for Plotting
ols_eval['residuals'] = ols_eval.ols_yhat - ols_eval.actual
validate_eval['residuals'] = validate_eval.ols_yhat - validate_eval.actual


NameError: name 'ols_eval' is not defined

In [ ]:
#  Compute the RMSE  and R2 for  ols model and baseline 
baseline_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))


In [10]:
# Create Model
ols_model3 = ols(formula='log_error ~ area_cluster_la_newer + area_cluster_la_older + area_cluster_northwest_costal + area_cluster_palmdale_landcaster + area_cluster_santa_clarita + area_cluster_se_coast + taxvaluedollarcnt + structuretaxvaluedollarcnt + taxvaluedollarcnt + landtaxvaluedollarcnt + acres + age', data=train).fit()

# Make predictions
ols3_yhat = ols_model3.predict(X_train)
ols_eval['ols3_yhat']=ols3_yhat
X_train['ols3_prdictions']=ols3_yhat


NameError: name 'ols_eval' is not defined

In [ ]:
#Calculate Evaluation metrics
#our RMSE baseline
baseline_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
#our model baseline
ols_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))
#olse (current) model baseline
ols3_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols3_yhat)))

pct_change_baseline=(((ols_RMSE-baseline_RMSE)/baseline_RMSE)*100)
pct_change_last_model=(((ols3_RMSE-ols_RMSE)/ols_RMSE)*100)
#rmse_validate = (sqrt(mean_squared_error(validate_eval.actual, validate_eval.ols2_yhat)))

baseline_r2 = (r2_score(ols_eval.actual, ols_eval.baseline_yhat))
ols3_train_r2 = (r2_score(ols_eval.actual, ols_eval.ols3_yhat))
#ols2_validate_r2 = (r2_score(validate_eval.actual, validate_eval.ols2_yhat))
# Display findings
print(f'My model has value: {ols_RMSE < baseline_RMSE}')
print(f'My model beats previous model: {ols3_RMSE < ols_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'Model 1 RMSE: {ols_RMSE}')
print(f'Current model train RMSE: {ols3_RMSE}')
#print(f'Currennt model validate RMSE: {rmse_validate}')
print()
print(f'Current model RMSE difference from baseline: {baseline_RMSE- ols_RMSE}')
#print(f'RMSE difference train to validate: {ols2_RMSE- rmse_validate}')
print(f'Current model baseline RMSE improvement: {pct_change_baseline}%')
#print(f'Current model RMSE improvement from last model: {pct_change_last_model}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Model train  R2: {ols3_train_r2}')
#print(f'Model Validate R2: {ols2_validate_r2}')

In [ ]:
features = [
'area_cluster_la_newer' , 'area_cluster_la_older' , 'area_cluster_northwest_costal' , 
'area_cluster_palmdale_landcaster' , 'area_cluster_santa_clarita' , 'area_cluster_se_coast' , 
'size_cluster_a' , 'size_cluster_b' , 'size_cluster_c' , 'size_cluster_d' , 'size_cluster_e' , 
'size_cluster_f' , 'size_cluster_g' , 'price_cluster_a' , 'price_cluster_b' , 'price_cluster_c' , 
'price_cluster_d', 'price_cluster_e']

In [ ]:
X_train01 = train[features]
X_validate01 = X_validate[features]

In [ ]:
y_validate.head()

In [ ]:
k=5
cluster_name = ‘tax_cluster’
cluster_vars = [‘taxamount’, ‘taxvaluedollarcnt’, ‘structuretaxvaluedollarcnt’, ‘landtaxvaluedollarcnt’]
# fit kmeans
kmeans = create_clusters(X_train, k, cluster_vars)
kmeans = create_clusters(X_validate, k, cluster_vars)
# get centroid values per variable per cluster
centroid_df = get_centroids(kmeans, cluster_vars, cluster_name)

In [ ]:
y_train['lars_yhat'] = lars.predict(X_train01)
rmse_train = mean_squared_error(y_train.logerror, y_train['lars_yhat'])**(1/2)
y_validate['1ars_yhat'] = lars.predict(X_validate01)
rmse_validate = mean_squared_error(y_validate.logerror, y_validate['1ars_yhat'])**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
#why is it the same as the baseline for my training in sample?

In [ ]:
glm = TweedieRegressor(power=1, alpha=0)
glm.fit(X_train01, y_train.logerror)

y_train['glm_yhat'] = glm.predict(X_train01)
rmse_train = mean_squared_error(y_train.logerror, y_train['glm_yhat'])**(1/2)
y_validate['glm_yhat'] = glm.predict(X_validate01)
rmse_validate = mean_squared_error(y_validate.log_error, y_validate['glm_yhat'])**(1/2)
print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
floats = (train.dtypes == train.drfloat64)

In [ ]:


# Make predictions
ols3_yhat = ols_model3.predict(X_train)
ols_eval['ols3_yhat']=ols3_yhat
X_train['ols3_prdictions']=ols3_yhat

In [ ]:
X_train.head()

In [ ]:
# Lets select the features we want for modeling
X_train1 = X_train[features]
X_validate1 = X_validate[features]
#X_test1 = X_test[features]

In [ ]:
y_train = y_train

In [ ]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [ ]:
from sklearn.linear_model import LinearRegression,LassoLars,TweedieRegressor
from sklearn.metrics import explained_variance_score

In [ ]:
# Created a linear regression model and appended the data into our metric_df dataframe
lm = LinearRegression(normalize=True)
# fitting the data
lm.fit(X_train, y_train.logerror)
#Predicting the values
y_train['logerror_pred_lm'] = lm.predict(X_train)
# appending the RMSE and R^2 to dataframe
#metric_df_train = modeling.make_metric_df_train(y_train.target, y_train.logerror_pred_lm, "lm_model", metric_df_train)

In [ ]:

#  DataFrame for evaluating model 
ols_eval = y_train.copy()
validate_eval = y_validate.copy()
ols_eval = pd.DataFrame(ols_eval)
validate_eval = pd.DataFrame(validate_eval)
ols_eval.rename(columns={'logerror': 'actual'}, inplace=True)
validate_eval.rename(columns={'logerror': 'actual'}, inplace=True)

In [ ]:
#ols_eval

In [ ]:
# Add baseline - median home value
ols_eval['baseline_yhat'] = ols_eval['actual'].mean()
validate_eval['baseline_yhat'] = ols_eval['actual'].mean()

In [ ]:
# Add model prediction
ols_eval['ols_yhat'] = ols_model1.predict(X_train)

In [ ]:
# Calculate and Add Residuals Column for Plotting
ols_eval['residuals'] = ols_eval.ols_yhat - ols_eval.actual
#validate_eval['residuals'] = validate_eval.ols_yhat - validate_eval.actual

In [ ]:
## Evaluation tools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

In [ ]:
#  Compute the RMSE  and R2 for  ols model and baseline 
baseline_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat)))
ols_RMSE = (sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat)))


In [ ]:
baseline_RMSE

In [ ]:
pct_change=round(((ols_RMSE-baseline_RMSE)/baseline_RMSE)*100, 2)
#rmse_validate = round(sqrt(mean_squared_error(validate_eval.actual, validate_eval.ols_yhat)))
baseline_r2 = (r2_score(ols_eval.actual, ols_eval.baseline_yhat), 2)
ols_train_r2 = (r2_score(ols_eval.actual, ols_eval.ols_yhat), 2)
#ols_validate_r2 = round(r2_score(validate_eval.actual, validate_eval.ols_yhat), 2)

#Output Findings

print(f'My model has value: {ols_RMSE < baseline_RMSE}')
print()
print(f'Baseline RMSE: {baseline_RMSE}')
print(f'My model train RMSE: {ols_RMSE}')
#print(f'My model validate RMSE: {rmse_validate}')
print(f'RMSE difference baseline to model: {baseline_RMSE- ols_RMSE}')
#print(f'RMSE difference train to validate: {ols_RMSE- rmse_validate}')
print(f'RMSE improvement: {pct_change}%')
print()
print(f'Baseline R2: {baseline_r2}')
print(f'Model train  R2: {ols_train_r2}')
#print(f'Model Validate R2: {ols_validate_r2}')

In [ ]:
#this is from our best of the four models we ran off of OLS
#likely no diff on models but run for practice

In [ ]:
# Create Model
lars_model1 = LassoLars(formula='log_error ~ area_cluster_la_newer + area_cluster_la_older + area_cluster_northwest_costal + area_cluster_palmdale_landcaster + area_cluster_santa_clarita + area_cluster_se_coast + size_cluster_a + size_cluster_b + size_cluster_c + size_cluster_d + size_cluster_e + size_cluster_f + size_cluster_g + price_cluster_a + price_cluster_b + price_cluster_c + price_cluster_d + price_cluster_e', data=train).fit()

# Make predictions
lars_yhat = lars_model1.predict(X_train)
X_train['mvp_prdictions']=lars_yhat

In [ ]:
# Create Model
#lars_model1 = lars(formula='log_error ~ area_cluster_la_newer + area_cluster_la_older + area_cluster_northwest_costal + area_cluster_palmdale_landcaster + area_cluster_santa_clarita + area_cluster_se_coast + size_cluster_a + size_cluster_b + size_cluster_c + size_cluster_d + size_cluster_e + size_cluster_f + size_cluster_g + price_cluster_a + price_cluster_b + price_cluster_c + price_cluster_d + price_cluster_e', data=train).fit()
features = ['area_cluster_la_newer' , 'area_cluster_la_older', 'area_cluster_northwest_costal',  'area_cluster_palmdale_landcaster',  'area_cluster_santa_clarita' , 'area_cluster_se_coast' , 'size_cluster_a' , 'size_cluster_b' , 'size_cluster_c' , 'size_cluster_d' , 'size_cluster_e' , 'size_cluster_f' , 'size_cluster_g' , 'price_cluster_a' , 'price_cluster_b' , 'price_cluster_c' , 'price_cluster_d' , 'price_cluster_e']



In [ ]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [ ]:

lars = LassoLars(alpha=1.0)
lars.fit(X_train[features], y_train['logerror'])

# Make predictions
lars_yhat = lars.predict(X_train[features])
X_train['mvp_lars_prdictions']=lars_yhat


In [ ]:
rmse_train = mean_squared_error(y_train.logerror, y_train['lars_yhat'])**(1/2)


y_validate['lars_yhat'] = lars.predict(X_validate)

rmse_validate = mean_squared_error(y_validate.logerror, y_validate['lars_yhat'])**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)


In [ ]:
X_train['area_cluster_la_newer' , 'area_cluster_la_older', 'area_cluster_northwest_costal',  'area_cluster_palmdale_landcaster',  'area_cluster_santa_clarita' , 'area_cluster_se_coast' , 'size_cluster_a' , 'size_cluster_b' , 'size_cluster_c' , 'size_cluster_d' , 'size_cluster_e' , 'size_cluster_f' , 'size_cluster_g' , 'price_cluster_a' , 'price_cluster_b' , 'price_cluster_c' , 'price_cluster_d' , 'price_cluster_e']


lars = LassoLars(alpha=1.0)
lars.fit(X_train[features], y_train['logerror'])

# Make predictions
lars_yhat = lars_model.predict(X_train[features])
X_train['mvp_lars_prdictions']=lars_yhat


In [ ]:
# lars = LassoLars(alpha=1.0)
# lars.fit(X_train, y_train['tax_value'])
# #evaluate: rmse
# y_train['lars_yhat'] = lars.predict(X_train)

# rmse_train = mean_squared_error(y_train.tax_value, y_train['lars_yhat'])**(1/2)


# y_validate['1ars_yhat'] = lars.predict(X_validate)

# rmse_validate = mean_squared_error(y_validate.tax_value, y_validate['1ars_yhat'])**(1/2)

# print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
#       "\nValidation/Out-of-Sample: ", rmse_validate)